In [ ]:
def trimDists():
  for off, reports in allWith('reports', offerings):
    newReports = []
    for report, responses in allWith('responses', reports[1:]):
      newResponses = {}
      for key, response in responses.iteritems():
        if 'distParams' in response:
          del responses[key]['distParams']
        if 'dist' in response:
          del responses[key]['dist']
        newResponses[key] = (response)
      report['responses'] = newResponses
      newReports.append(report)
    off['reports'] = newReports
    
    
trimDists()    
big = [o for o in offerings if o['objectID'] == '121820_11390'][0]
print json.dumps(big['reports'][0], indent=2)

sizes = [float(r['size']) for r in filteredReports]
lessThanTen = [r['size'] < 25 for r in filteredReports]
scores = [float(r['responses']['overall']['score']) for r in filteredReports]
xRows = [sizes, lessThanTen]
xs = [[row[i] for row in xRows] for i in range(0, len(xRows[0]))]

findAndDoElse = lambda fn, _else: findAndDo(lambda x: fn(x) if x else _else)
findAndGetFloat = findAndDoElse(lambda x: float(x), 0.0)
findAndGet = findAndDo(lambda x:x)
findAndCompFloat = lambda string: findAndDoElse(lambda x: float(x == string), 0.0)
allWith = lambda keyPath, lst:[x for x in lst if findAndGet]

import utilities
profReports = []
xVarsDict = {}
filteredReports = [r for r in reports if findAndGetFloat(r, 'responses/overall/score') > 1.0][0:400]
print len(filteredReports)

pastReports = []
for _, courseReports in qLookup.items()[0:100]:
  oldOnes = [r for r in courseReports if int(r['year']) < 2014]
  oldOnes.sort(key=itemgetter('term'))
  oldOnes.sort(key=itemgetter('year'), reverse=True)
  if len(oldOnes) > 1:
    for i, pastReport in enumerate(oldOnes):
      pastReports.append((pastReport, oldOnes[i + 1:]))      

class xVar():
  def __init__(self, label, fn):
    self.label = label
    self.fn = fn

def makeProfVar(prof, profVars):
  label = 'Prof: ' + prof['displayName']
  if label not in profVars:
    xVars.append(xVar(label, lambda report: findAndCompFloat(prof['matchName'])(report, 'profs/[0]/matchName')))
    profVars[label] = True

def makeLevelVar(level, levelVars):
  label = 'Level: ' + level
  if label not in profVars:
    xVars.append(xVar(label, lambda report: findAndCompFloat(level)(report, 'level')))
    levelVars[label] = True

def makeGroupVar(group, groupVars):
  label = 'Group: ' + group
  if label not in groupVars:
    xVars.append(xVar(label, lambda report: findAndCompFloat(group)(report, 'group')))
    groupVars[label] = True

xVars = []
groupVars = {}
profVars = {}
levelVars = {}

xVars.append(xVar('Size^0.5: ', lambda report: float(report['size']) ** 0.5))
xVars.append(xVar('Size^0.25: ', lambda report: float(report['size']) ** 0.25))

# xVars.append(xVar('Prof: None', lambda report: float(findAndGet(report, 'profs/[0]/responses/instructor/score') is None)))

def parseUndergrad(report):
  matched = re.compile('\d\d*').findall(report['number'])
  numInt = int(matched[0]) if len(matched) > 0 else -1
  if numInt < 300:
    return 1.
  elif numInt >= 300 and numInt < 1000:
    return 0.
  elif numInt >= 1000 and numInt < 3000:
    return 1.
  else:
    return 0.
  
xVars.append(xVar('isUndergrad: ', parseUndergrad))

# for report in filteredReports:
#   if 'group' in report:
#     makeGroupVar(report['group'], groupVars)

# for report in filteredReports:
#   if 'profs' in report:
#     makeProfVar(report['profs'][0], profVars)

import numpy as np
import pandas as pd
import statsmodels.api as sm
import math
from sklearn.preprocessing import StandardScaler
import statsmodels.graphics.regressionplots as rplt

X = np.array([np.array([var.fn(report) for var in xVars]) for report in filteredReports])
scaler = StandardScaler()
scaler.fit(X)
# X = scaler.transform(X)
X = sm.add_constant(X)
y = np.array([findAndGetFloat(report, 'responses/overall/score') for report in filteredReports])

model = sm.OLS(y, X)
result = model.fit()

print 'Intercept: ', result.params[0]
print 'R^2: ', result.rsquared
for var, coef in zip(xVars, result.params[1:]):
  print var.label, coef

result.summary()

rplt.plot_fit(result, 3)

# fakeReports = [{'size': x} for x in [1, 10, 25, 50, 100, 200, 500]]
# fakeX = np.array([np.array([var.fn(report) for var in xVars]) for report in fakeReports])
# scaler.fit(fakeX)
# fakeX = scaler.transform(fakeX)
# fakeX = sm.add_constant(fakeX)
# result.predict(fakeX)

def indicator(index):
  def tester(report, past):
    return float(len(past) >= index + 1)
  return tester
    
def getter(attribute, index):
  def get(report, past):
    if len(past) < index + 1:
      return 4.1
    val = findAndDo(lambda x:x)(past[index], 'responses/' + attribute + '/score')
    return float(val) if val else 4.1
  return get
  
for index in [1, 2, 3]:
  xVars.append(xVar('Most recent indicator ' + str(index) + ':', indicator(index)))
  for attribute in ['overall', 'workload', 'recommend', 'section', 'feedback', 'materials']:
    xVars.append(xVar('  ' + attribute + '_' + str(index), getter(attribute, index)))  
  
xVars.append(xVar('Size: ', lambda report, _: findAndDo(lambda x: x)(report, 'size')))
xVars.append(xVar('Evals: ', lambda report, _: findAndDo(lambda x: x)(report, 'evals')))

for report, past in pastReports:
  if 'group' in report:
    makeGroupVar(report['group'])

from sklearn.neural_network import MLPRegressor
xRows = []
ys = []

for report, past in pastReports:
  xRows.append([var.fn(report, past) for var in xVars])
  ys.append(findAndDo(lambda x: float(x))(report, 'responses/overall/score'))

clf = MLPRegressor(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(40, 15), 
                    random_state=2)
import numpy as np
# ys = np.array([1, 2, 3]).reshape(-1, 1)
reg = clf.fit(xRows, ys)

# for xVar, coef in zip(xVars, reg.coef_):
#   print xVar.label, coef
error = 0
for report, _ in pastReports:
  xRow = [var.fn(report, past) for var in xVars]
  prediction = reg.predict(xRow)
  realScore = findAndDo(lambda x: float(x))(report, 'responses/overall/score')
  error += abs(prediction - realScore)
#   print 'Real vs predicted: ', '{0:0.2f}'.format(realScore), '{0:0.2f}'.format(prediction[0])
print 'Mean error:', error / len(pastReports)
print len(xRows)
reg.coefs_[0]
  